In [ ]:
from datetime import datetime
import requests
from io import BytesIO
from PIL import Image

import pandas as pd
import numpy as np

import nfl_data_py as nfl

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import plotly.colors as cl

from resources.plotly_theme import nfl_template
from resources.heat_map import heat_map
from resources.get_nfl_data import get_pbp_data, get_team_info

from resources.team_stats import get_team_stats

pio.templates['nfl_template'] = nfl_template

In [ ]:
''' Parameters '''

WEEK = 13


''' Import Data '''

# Import
team_data = get_team_info()
pbp_data = get_pbp_data(years=[2025])

league_data = pbp_data.loc[pbp_data['week'] < WEEK, :].copy()

player_info = nfl.import_players()


In [ ]:
''' Next Gen Stats '''

ngs = nfl.import_ngs_data(years=[2025], stat_type='passing')

print(ngs.head().to_string())

In [ ]:
''' FTN '''

ftn = nfl.import_ftn_data(years=[2025])
ftn = ftn.loc[ftn['week'] < WEEK, :]

# nfl.import_pbp_data



print(ftn.shape)
print(ftn.columns)
print(ftn.head(20).to_string())

In [ ]:
print(league_data.loc[(league_data['game_id'] == '2025_01_ARI_NO') & (league_data['play_id'] == 85), :].to_string())
print(ftn.loc[(ftn['nflverse_game_id'] == '2025_01_ARI_NO') & (ftn['nflverse_play_id'] == 85), :].to_string())

In [ ]:
league_data_ftn = league_data.merge(ftn, left_on=['game_id', 'play_id'], right_on=['nflverse_game_id', 'nflverse_play_id'],
                                    how='left')

print(league_data.shape)
print(league_data_ftn.shape)

In [ ]:
qb_loc = league_data_ftn.loc[league_data_ftn['qb_location'] != '0', :].groupby(['posteam', 'qb_location']).aggregate(
    Plays=('qb_location', 'size'),
    Pass=('pass', 'sum'),
    Rush=('rush', 'sum'),
    Yards=('yards_gained', 'sum'),
    PassYards=('passing_yards', 'sum'),
    RushYards=('rushing_yards', 'sum'),
)
qb_loc['% Plays'] = qb_loc['Plays'] / qb_loc.groupby(level=0)['Plays'].transform(np.sum)


print(qb_loc.head(15).to_string())

In [ ]:

bar = px.bar(
    x=qb_loc['% Plays'].tolist(),
    y=qb_loc.index.get_level_values(0),
    color=qb_loc.index.get_level_values(1)
)

bar.show()

### "pass" and "rush"
 - *Designed* pass or rush, regardless of outcome
   - QB scrambles, sacks, penalties, blown dead, etc.

### dropback
 - Non-penalty designed pass plays
 - INCLUDES eventual qb scrambles

### Sacks
 - Count as dropback, pass_attempt, pass, and sack

### Rushing
 - per "Designed Rush"
    - EPA per Designed Rush
    - Success Rate on Designed Rushes
    - where rush == 1

### Passing
 - per "Dropback"
    - EPA per Dropback
    - Success Rate on Dropbacks
    - where pass == 1
    - includes QB scrambles, which are rush attempts
